<a href="https://colab.research.google.com/github/gjhernandezp/algorithms/blob/master/Python/Quantiacs/quantiacscolab_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://quantiacs.com/documentation/en/user_guide/local_development.html#google-colab-support

# The environment

The next three code cells contains commands that will configure your Coogle Colab environment. 

When you transfer the strategy to Quantiacs, remove these cells.

They are not relevant for the Quantiacs platform.

In [1]:
!pip uninstall xarray -y
!pip install xarray==0.19.0

Found existing installation: xarray 0.20.2
Uninstalling xarray-0.20.2:
  Successfully uninstalled xarray-0.20.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827 kB 5.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires xarray>=0.20, but you have xarray 0.19.0 which is incompatible.


At first, setup the toolbox from github using pip:

In [ ]:
###DEBUG###

! pip install git+https://github.com/quantiacs/toolbox.git

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/quantiacs/toolbox.git to /tmp/pip-req-build-eacv8538
  Running command git clone -q https://github.com/quantiacs/toolbox.git /tmp/pip-req-build-eacv8538
     |████████████████████████████████| 9.9 MB 4.3 MB/s 
     |████████████████████████████████| 355 kB 61.1 MB/s 
     |████████████████████████████████| 3.4 MB 45.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.8 MB/s 
  Created wheel for qnt: filename=qnt-0.0.272-py3-none-any.whl size=114264 sha256=1c47e49733c14a072d5af8fcf3b0810a0ee5d9152286cbba03ebb462801bee55
  Stored in directory: /tmp/pip-ephem-wheel-cache-_37vi8dt/wheels/eb/4b/a6/2941ec3b0a208a66ffa127c74cbcddbff9453f5ed798ee9be2
Successfully built qnt
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:


Then install TA-Lib (indicators library) if you need it.

Instead of TA-Lib you can use qnt.ta or another library. In this case, skip the next cell.

Note that the installation can take several minutes.

In [ ]:
###DEBUG###
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

Finally specify the api key and disable interactive charts.

You can find the api key in [your profile](https://quantiacs.com/personalpage/homepage).

We disable interactive charts in the library because interact+plotly do not work correctly in Google Colab.

In [ ]:
###DEBUG###
import os

os.environ['API_KEY'] = "" 
os.environ['NONINTERACT'] = 'True' 

# The strategy

The next cell contains the strategy code itself.

In [ ]:

import xarray as xr

import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.graph as qngraph


def load_data(period):
    return qndata.stocks.load_ndx_data(tail=period)



def strategy(data):
    close     = data.sel(field="close")
    is_liquid = data.sel(field="is_liquid")
    sma_slow  = qnta.sma(close, 200).isel(time=-1)
    sma_fast  = qnta.sma(close, 20).isel(time=-1)
    weights   = xr.where(sma_slow < sma_fast, 1, -1)
    weights   = weights * is_liquid
    return weights



weights = qnbt.backtest(
    competition_type = "stocks_nasdaq100",
    load_data        = load_data,
    lookback_period  = 365*4,
    start_date       = "2020-10-10",
    strategy         = strategy,
    analyze          = True,
    build_plots      = True
)

#Submit your strategy

Finally, you can transfer this notebook to Quantiacs.

Just create a new strategy notebook in [your area](https://quantiacs.com/personalpage/strategies) and upload this notebook to this new environment (replace `strategy.ipynb`).

Don't forget to **remove code cells from "The environment"** section.

If you have any special external dependency which is absent in Quantiacs jupyter environment, add their installation code to `init.ipynb`.

Then test and submit your strategy.